<a href="https://colab.research.google.com/github/vinu-pillai/mlops_poc/blob/main/onnx_to_aml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install azure-ai-ml azure-identity onnx
import os
import shutil
import hashlib
from datetime import datetime, timezone
from pathlib import Path
from typing import Optional

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model

import onnx


# ========= USER INPUTS =========
# Required: path to your ONNX model file
ONNX_FILE = Path("linear_regression.onnx")

# Optional companion files (if present they will be packaged with the model)
OPTIONAL_FILES = [Path("params.json"), Path("MODEL_CARD.md")]

# Model naming
MODEL_NAME = "linear-regression-onnx"

# Target: set REGISTRY_NAME to register into an AML Registry;
# or leave REGISTRY_NAME empty to register into a specific Workspace.
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID", "<sub-id>")
RESOURCE_GROUP  = os.getenv("AZ_RESOURCE_GROUP", "<rg-name>")
WORKSPACE_NAME  = os.getenv("AZ_ML_WORKSPACE",  "<workspace-name>")
REGISTRY_NAME   = os.getenv("AZ_ML_REGISTRY",   "")  # e.g., "company-registry"; if non-empty, will use Registry

# Staging folder to bundle files for upload
STAGING_DIR = Path("model_artifact_staging")
# =================================




def get_ml_client(
INPUT AZURE PLATFORM DETAILS
                   ) -> MLClient: Update with code to Authenticate

def register_model():
    if not ONNX_FILE.exists():
        raise FileNotFoundError(f"ONNX file not found: {ONNX_FILE.resolve()}")

    version = compute_version(ONNX_FILE)
    ir_version, opsets = extract_onnx_meta(ONNX_FILE)
    artifact_dir = build_staging_dir(ONNX_FILE, OPTIONAL_FILES, STAGING_DIR)

    # Compose tags & properties helpful for discovery / governance
    tags = {
        "framework": "onnx",
        "task": "regression",
        "artifact_hash": sha256_of_file(ONNX_FILE)[:12],
        "onx_ir": str(ir_version or ""),
        **{f"opset:{k}": str(v) for k, v in opsets.items()},
    }
    props = {
        "primary_file": ONNX_FILE.name,
        "packaging": "custom_model",
        "source": "script_upload",
    }

    ml = get_ml_client(SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE_NAME, REGISTRY_NAME)

    model = Model(
        name=MODEL_NAME,
        version=version,
        path=str(artifact_dir),     # directory with onnx + extras
        type="custom_model",        # raw ONNX (no MLflow flavor)
        description="Linear regression ONNX model with optional params.json and model card.",
        tags=tags,
        properties=props,
    )

    created = ml.models.create_or_update(model)
    print("\n=== Model Registered ===")
    print(f"Name     : {created.name}")
    print(f"Version  : {created.version}")
    print(f"ID       : {created.id}")             # ARM id (workspace) or registry path
    print(f"Path     : {created.path}")
    print(f"Type     : {created.type}")
    print(f"Tags     : {created.tags}")
    print(f"Props    : {created.properties}")

    if REGISTRY_NAME:
        print(f"\nRegistered to Registry: '{REGISTRY_NAME}'.")
    else:
        print(f"\nRegistered to Workspace: '{WORKSPACE_NAME}' (RG: {RESOURCE_GROUP}).")

    print("\nTip: You can now deploy this model to a Managed Online Endpoint or reference it in pipelines by name+version.")


if __name__ == "__main__":
    register_model()
